In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import pickle
import h5py

In [24]:
HDF5_DB = '../datasets/animals/hdf5/features.hdf5_'
MODEL = 'animals.cpickle'
JOBS = -1

In [25]:
# Open the HDF5 database for reading then determine the index of the training and testing
# split, provided that this data was already shuffled prior to writing it to disk
db = h5py.File(HDF5_DB, "r")
i = int(db["labels"].shape[0] * 0.75)

In [26]:
db.__contains__('features')
[key for key in db.keys()]

['labels']

In [5]:
# Define the set of parameters that we want to tune then start a grid search 
# where we evaluate our model for each value of C
print("[INFO] Tuning Hyperparameters")
params = {"C": [0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]}

model = GridSearchCV(LogisticRegression(), params, cv=3, n_jobs=JOBS)
model.fit(db["features"][:i], db["labels"][:i] )

print("[INFO] Best Hyperparamerters: {}".format(model.best_params_))

[INFO] Tuning Hyperparameters


KeyError: "Unable to open object (object 'features' doesn't exist)"